<a href="https://colab.research.google.com/github/Student-PIAIC/Crew_AI/blob/main/Crewai%2C_Augmented_llm%2Cmemory%2CCrew_and_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY_2')


In [ ]:
!pip install -Uq crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.5/548.5 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 23.8 MB/s 

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from crewai.flow.flow import Flow, start, listen
class MyFlow(Flow):
  @start()
  def function(self):
    print("step...1")

  @listen(function)
  def function1(self):
    print("step...2")
obj=MyFlow()
obj.kickoff()

 
[2025-03-09 18:14:05][🌊 FLOW CREATED: 'MYFLOW']: 2025-03-09 18:14:05.262489
 
[2025-03-09 18:14:05][🤖 FLOW STARTED: 'MYFLOW', B5DDE855-FAC5-48A5-B6D7-EBFC861E3168]: 2025-03-09 18:14:05.264655
 Flow started with ID: b5dde855-fac5-48a5-b6d7-ebfc861e3168
 
[2025-03-09 18:14:05][🤖 FLOW METHOD STARTED: 'FUNCTION']: 2025-03-09 18:14:05.265233
step...1
 
[2025-03-09 18:14:05][👍 FLOW METHOD FINISHED: 'FUNCTION']: 2025-03-09 18:14:05.265332
 
[2025-03-09 18:14:05][🤖 FLOW METHOD STARTED: 'FUNCTION1']: 2025-03-09 18:14:05.265532
step...2
 
[2025-03-09 18:14:05][👍 FLOW METHOD FINISHED: 'FUNCTION1']: 2025-03-09 18:14:05.265602
 
[2025-03-09 18:14:05][👍 FLOW FINISHED: 'MYFLOW', B5DDE855-FAC5-48A5-B6D7-EBFC861E3168]: 2025-03-09 18:14:05.265730


In [ ]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY_2')
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
    api_key=os.environ['GOOGLE_API_KEY']
)

llm1.call("What is the capital of Pakistan?")


 
[2025-03-09 18:22:05][🤖 LLM CALL STARTED]: 2025-03-09 18:22:05.488055
 
[2025-03-09 18:22:05][✅ LLM CALL COMPLETED]: 2025-03-09 18:22:05.928825


'The capital of Pakistan is **Islamabad**.\n'

In [ ]:
google_embedder = {
    "provider": "google",
    "config": {
        "model": "models/text-embedding-004",
            "api_key": userdata.get('GOOGLE_API_KEY_2'),
         }
}

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Create a knowledge source
content = "Users name is Asif. He is 35 years old and lives in Karachi."
string_source = StringKnowledgeSource(
    content=content,
)

# Create an LLM with a temperature of 0 to ensure deterministic outputs
llm = llm1

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source], # Enable knowledge by adding the sources here. You can also add more sources to the sources list.
    embedder=google_embedder
)

result = crew.kickoff(inputs={"question": "What city does Asif live in and how old is he?"})

 
[2025-03-09 18:22:21][🚀 CREW 'CREW' STARTED, 50DD8468-0183-42F1-93AF-59CA7CAAE0AB]: 2025-03-09 18:22:21.928997
 
[2025-03-09 18:22:21][📋 TASK STARTED: ANSWER THE FOLLOWING QUESTIONS ABOUT THE USER: WHAT CITY DOES ASIF LIVE IN AND HOW OLD IS HE?]: 2025-03-09 18:22:21.940726
 
[2025-03-09 18:22:22][🤖 AGENT 'ABOUT USER' STARTED TASK]: 2025-03-09 18:22:22.667481
# Agent: About User
## Task: Answer the following questions about the user: What city does Asif live in and how old is he?
 
[2025-03-09 18:22:22][🤖 LLM CALL STARTED]: 2025-03-09 18:22:22.667869
 
[2025-03-09 18:22:23][✅ LLM CALL COMPLETED]: 2025-03-09 18:22:23.249918


# Agent: About User
## Final Answer: 
Asif lives in Karachi and he is 35 years old.


 
[2025-03-09 18:22:23][✅ AGENT 'ABOUT USER' COMPLETED TASK]: 2025-03-09 18:22:23.251603
 
[2025-03-09 18:22:23][✅ TASK COMPLETED: ANSWER THE FOLLOWING QUESTIONS ABOUT THE USER: WHAT CITY DOES ASIF LIVE IN AND HOW OLD IS HE?]: 2025-03-09 18:22:23.251797
 
[2025-03-09 18:22:23][✅ 

In [ ]:
import os
os.environ['CREWAI_STORAGE_DIR']='/my_crew1'


In [ ]:
from crewai import LLM
llm1=LLM(
    model="gemini/gemini-2.0-flash"
)

In [ ]:

from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
# from crewai.memory.storage import LTMSQLiteStorage, RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage

from typing import List, Optional

agent=Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task=Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)
crew = Crew(
    agents = [agent],
    tasks = [task],
    process = Process.sequential,
    memory = True,
    # Long-term memory for persistent storage across sessions
    long_term_memory = LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="./my_crew2/long_term_memory_storage.db"
        )
    ),
    # Short-term memory for current context using RAG
    short_term_memory = ShortTermMemory(
        storage = RAGStorage(
                embedder_config=google_embedder,
                type="short_term",
                path="./my_crew2/short_term"
        ),
    ),
    # Entity memory for tracking key information about entities
    entity_memory = EntityMemory(
        storage=RAGStorage(
            embedder_config=google_embedder,
            type="short_term",
            path="./my_crew2/entity"
        )
    ),
    verbose=True,
)


In [ ]:
import os
from crewai import Crew, Process
from mem0 import MemoryClient
from crewai.memory import UserMemory  # Import UserMemory

# Set environment variables for Mem0
os.environ["MEM0_API_KEY"] = userdata.get('MEM0_API_KEY')

# Step 1: Record preferences based on past conversation or user input
client = MemoryClient()
messages = [
    {"role": "user", "content": "Hi there! I'm planning a vacation and could use some advice."},
    {"role": "assistant", "content": "Hello! I'd be happy to help with your vacation planning. What kind of destination do you prefer?"},
    {"role": "user", "content": "I am more of a beach person than a mountain person."},
    {"role": "assistant", "content": "That's interesting. Do you like hotels or Airbnb?"},
    {"role": "user", "content": "I like Airbnb more."},
]
client.add(messages, user_id="john")

# Step 2: Create a Crew with User Memory

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    verbose=True,
    memory=True,
    memory_config={
        "provider": "mem0",
        "config": {"user_id": "john"},
    },

)

crew.kickoff(inputs={"question": "What is your favorite vacation destination?"})

 
[2025-03-09 18:37:39][🚀 CREW 'CREW' STARTED, 21E7D738-BF3F-4122-90C9-A39EB5BD6BE7]: 2025-03-09 18:37:39.910144
 
[2025-03-09 18:37:39][📋 TASK STARTED: ANSWER THE FOLLOWING QUESTIONS ABOUT THE USER: WHAT IS YOUR FAVORITE VACATION DESTINATION?]: 2025-03-09 18:37:39.924498
 
[2025-03-09 18:37:40][❌ TASK FAILED: ANSWER THE FOLLOWING QUESTIONS ABOUT THE USER: WHAT IS YOUR FAVORITE VACATION DESTINATION?]: 2025-03-09 18:37:40.107233
 
[2025-03-09 18:37:40][❌ CREW 'CREW' FAILED, 21E7D738-BF3F-4122-90C9-A39EB5BD6BE7]: 2025-03-09 18:37:40.107346


AttributeError: 'NoneType' object has no attribute 'search'

In [ ]:
!pip install -Uq 'crewai[tools]'

In [ ]:
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')

In [ ]:
google_embedder = {
    "provider": "google",
    "config": {
        "model": "models/text-embedding-004",
            "api_key": userdata.get('GOOGLE_API_KEY_2'),
         }
}

In [ ]:
llm1 = LLM(
    model="gemini/gemini-2.0-flash"
)

In [ ]:
import os
from crewai import Agent, Task, Crew
from crewai_tools import(
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys
os.environ["SERPER_API_KEY"] = ("SERPER_API_KEY") # serper.dev API key
os.environ["GOOGLE_API_KEY"] = ("GOOGLE_API_KEY_2")

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the AI industry, based on the research analyst’s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)

# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True,  # Enable planning feature
    planning_llm=llm1
)

# Execute tasks
crew.kickoff()

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

KeyError: 'OPENAI_API_KEY'

In [ ]:
# from crewai.tools import tool
# @tool("PIAIC fee update")
# def my_tool(roll_num: int) -> dict | str:
#   """This function search piaic student fee status. It will reqfuire roll number of piaic student"""
#   data = {100:'paid',
#         200:'unpaid'}

#   if status:
#       return {"status": status}
#   else:
#       return "status not found"


from crewai.tools import tool

@tool("PIAIC fee update")
def my_tool(roll_no: int) -> dict | str:
    """this function search piaic student fee updates, it will required roll no of PIAIC student"""
    #database

    data = {100:'paid',
         200:'unpaid'}


    status = data.get(roll_no)

    if status:
      return {"status": status}
    else:
      return "student not found"





In [ ]:
from crewai import Agent, Task, Crew
from google.colab import userdata
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
    api_key=userdata.get("GOOGLE_API_KEY_2")
)
tool = my_tool

# Create agents
piaic_manager = Agent(
    role="PIAIC manager",
    goal = "Manage all quries regarding PIAIC and you will use only relevant tools for student query",
    backstory="""You are a master at understanding people and their preferences.""",
    tools=[my_tool],
    verbose=True,
    llm=llm1
)

piaic_card_creator = Task(
    description="you will be responsible for all PIAIC relevant operations, student query '{query}' you must be know how to answer his question based on final context",
    expected_output="final query answer only",
    agent=piaic_manager
)

crew = Crew(
    agents=[piaic_manager],
    tasks=[piaic_card_creator],
    verbose=True
)

result = crew.kickoff(inputs={
    "query":"I'm PIAIC student my name is Muhammad Asif and my roll number is 100, can you provide my fee status."
})

print(result)

 
[2025-03-09 18:45:02][🚀 CREW 'CREW' STARTED, 065B9707-E367-4E6B-AF94-9884882257C6]: 2025-03-09 18:45:02.327740
 
[2025-03-09 18:45:02][📋 TASK STARTED: YOU WILL BE RESPONSIBLE FOR ALL PIAIC RELEVANT OPERATIONS, STUDENT QUERY 'I'M PIAIC STUDENT MY NAME IS MUHAMMAD ASIF AND MY ROLL NUMBER IS 100, CAN YOU PROVIDE MY FEE STATUS.' YOU MUST BE KNOW HOW TO ANSWER HIS QUESTION BASED ON FINAL CONTEXT]: 2025-03-09 18:45:02.342946
 
[2025-03-09 18:45:02][🤖 AGENT 'PIAIC MANAGER' STARTED TASK]: 2025-03-09 18:45:02.344072
# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student my name is Muhammad Asif and my roll number is 100, can you provide my fee status.' you must be know how to answer his question based on final context
 
[2025-03-09 18:45:02][🤖 LLM CALL STARTED]: 2025-03-09 18:45:02.344269
 
[2025-03-09 18:45:03][✅ LLM CALL COMPLETED]: 2025-03-09 18:45:03.372799
 
[2025-03-09 18:45:03][🤖 TOOL USAGE STARTED: 'PIAIC FEE UPDATE'